In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import joblib
from src.libs.utils.functions import *

In [4]:
countries_req=requests.get('https://api.mercadolibre.com/sites')
countries=countries_req.json()
get_values_by_key(dictionary_list=countries,key_name='id')

['MPY',
 'MSV',
 'MLM',
 'MBO',
 'MLA',
 'MNI',
 'MHN',
 'MCU',
 'MLU',
 'MRD',
 'MCO',
 'MLC',
 'MEC',
 'MPE',
 'MGT',
 'MLB',
 'MCR',
 'MPA',
 'MLV']

In [31]:
cats = requests.get('https://api.mercadolibre.com/sites/MLA/categories')
categorias=cats.json()
categorias

[{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
 {'id': 'MLA1512', 'name': 'Agro'},
 {'id': 'MLA1403', 'name': 'Alimentos y Bebidas'},
 {'id': 'MLA1071', 'name': 'Animales y Mascotas'},
 {'id': 'MLA1367', 'name': 'Antigüedades y Colecciones'},
 {'id': 'MLA1368', 'name': 'Arte, Librería y Mercería'},
 {'id': 'MLA1743', 'name': 'Autos, Motos y Otros'},
 {'id': 'MLA1384', 'name': 'Bebés'},
 {'id': 'MLA1246', 'name': 'Belleza y Cuidado Personal'},
 {'id': 'MLA1039', 'name': 'Cámaras y Accesorios'},
 {'id': 'MLA1051', 'name': 'Celulares y Teléfonos'},
 {'id': 'MLA1648', 'name': 'Computación'},
 {'id': 'MLA1144', 'name': 'Consolas y Videojuegos'},
 {'id': 'MLA1500', 'name': 'Construcción'},
 {'id': 'MLA1276', 'name': 'Deportes y Fitness'},
 {'id': 'MLA5726', 'name': 'Electrodomésticos y Aires Ac.'},
 {'id': 'MLA1000', 'name': 'Electrónica, Audio y Video'},
 {'id': 'MLA2547', 'name': 'Entradas para Eventos'},
 {'id': 'MLA407134', 'name': 'Herramientas'},
 {'id': 'MLA1574', 'name': 'H

In [6]:
cat_id = 'MLA1000'
offset = 50
url = f'https://api.mercadolibre.com/sites/MLA/search?category={cat_id}&offset={offset}'
itm = requests.get(url)
items=itm.json()

In [10]:
requests.get('https://api.mercadolibre.com/sites/MLA/users/')

<Response [404]>